## import the data from the database 

This is my attempt to make a connection between the postgres database and this file
It seems to work up until the import config from config portion 
I'm not sure if I don't have my environment set up correctly or what the issue is 
but it is needed because the program will recognize the class we are trying to use as 
a module


In [1]:
#import the password from a seperate file that is ignored on github
# from config import password
# import dependencies 
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import LogisticRegression


In [ ]:
from getpass import getpass 
password = getpass ('Enter database password')

try: 
    conn = psycopg2.connect(
        host="localhost",
        database = "Tennessee_Tornadoes",
        user="postgres",
        password=password)
    print ('pyscopg2 connection:', conn)
    
except Exception as err:
    print ('psycopg2 connect() ERROR:', err)
    connect = None


In [ ]:
cr = conn.cursor()
cr.execute('SELECT * FROM cleaned_tn_tornadoes;')
tmp = cr.fetchall()

In [ ]:
#extract the  column names
col_names = []
for db in cr.description:
    col_names.append(db[0])

In [ ]:
# Create the dataframe, passing in the list of col_names extracted from the description
df = pd.DataFrame(tmp, columns=col_names)
df.head()

### Run the first model

target = loss

yields approx 55% accuracy 

In [ ]:
# prepare the dataframe 

df_3 = df.drop(['Unnamed: 0','Unnamed: 0.1','TIME',"WID","starting county",
              'SLAT','SLON','ELAT','ELON','LEN','ending county','FID'], axis=1)
df.columns


target = 'Property Loss'
X = pd.get_dummies(df_3.drop([target],axis = 1))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

from sklearn.datasets import fetch_covtype
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(
    MinMaxScaler(),
    PolynomialCountSketch(degree=2, n_components=300),
    LogisticRegression(max_iter=1000),
)

pipe.fit(X_train, y_train).score(X_test, y_test)


print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")
# Print the imbalanced classification report
print(f"Classification Report:\n{classification_report_imbalanced(y_test, y_pred)}")



### Run the second model 

target = Month 

yields approx 54% accuracy 

In [ ]:
#sklearn.linear_model.SGDClassifier

df_1 = df.drop(['Unnamed: 0','Unnamed: 0.1','TIME',
              'SLAT','SLON','ELAT','ELON','LEN','ending county','Injuries','FID'], axis=1)
# Create our features

target = 'MO'
X = pd.get_dummies(df_1.drop(columns=target))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

model = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
model.fit(X_train, y_train)

# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")
# Print the imbalanced classification report
print(f"Classification Report:\n{classification_report_imbalanced(y_test, y_pred)}")

### Run the third model 

target = Magnitude 


yields approx 59% accuracy 

Prepare the dataset 

In [ ]:
#Keep Date, Starting county, property loss and mag

df_2 = df.drop([ 'MO','Unnamed: 0','Unnamed: 0.1','YR','TIME',
              'SLAT','SLON','ELAT','ELON','LEN','ending county','starting county','Injuries','Fatalities','FID'], axis=1)

In [ ]:
# Create our features

target = 'MAG'
X = pd.get_dummies(df_2.drop(columns=target))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)


ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

pipe = make_pipeline(
    StandardScaler(),
    MinMaxScaler(),
    PolynomialCountSketch(degree=2, n_components=300),
    LogisticRegression(),
)

pipe.fit(X_resampled, y_resampled).score(X_resampled, y_resampled)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")